In [1]:
from tensorflow import keras
import tensorflow as tf
import tensorflow_hub as hub
import os
import tensorflow_text as tftext

import json
import math
import os
import csv
import collections
import pandas as pd

from absl import app
from absl import flags
from absl import logging


In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

In [3]:
##restricting no of gpus
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = gpus[3]
tf.config.experimental.set_memory_growth(device_to_use,True)
tf.config.experimental.set_visible_devices(device_to_use, 'GPU')
print(tf.config.experimental.get_visible_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]


In [4]:
##setting direcory for downloading tfhub modules
os.environ["TFHUB_CACHE_DIR"] = '/space/engineering/tfhub_modules'

In [5]:
vocab_file="/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt"

In [6]:
dish_small_spm_file='/var/extra/users/jgeorge/git/tensorflow_repos/sentencepiece/build/models/spm_dish.model'
dish_small_spm_vocab='/var/extra/users/jgeorge/git/tensorflow_repos/sentencepiece/build/models/spm_dish.vocab'

In [7]:
bert_model_dir='/space/engineering/pretrained_models/albert/albert_large'

In [8]:
bert_hub_url='https://tfhub.dev/tensorflow/albert_en_large/1'

In [9]:
bert_hub_loaded = hub.load(bert_hub_url)
sentencepiece_path = bert_hub_loaded.sp_model_file.asset_path.numpy()

In [10]:
##loading the sentence piece model into memory
sp_model_proto = tf.io.gfile.GFile(sentencepiece_path, 'rb').read()

In [11]:
sp3 = tftext.SentencepieceTokenizer(model=sp_model_proto)

In [12]:
sp_tokenizer = tftext.SentencepieceTokenizer(model=sp_model_proto)

In [14]:
sp3.id_to_string([0, 1, 2])

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'<pad>', b'<unk>', b'[CLS]'], dtype=object)>

In [13]:
actual_sentence = 'give me directions to nearest restaurant randomtext 15-08-2019 year 2019'
encoded_text3 = sp3.tokenize(actual_sentence)
print('encoded_text3 ',encoded_text3, ', decoded text3 ',sp3.detokenize(encoded_text3.numpy()),'\n')

encoded_text3  tf.Tensor(
[  590    55  7876    20  6557  3284  5477 11969   357     8  3099     8
  1323  1433   159 10114], shape=(16,), dtype=int32) , decoded text3  tf.Tensor(b'give me directions to nearest restaurant randomtext 15-08-2019 year 2019', shape=(), dtype=string) 



In [24]:
encoded_text3

<tf.Tensor: shape=(16,), dtype=int32, numpy=
array([  590,    55,  7876,    20,  6557,  3284,  5477, 11969,   357,
           8,  3099,     8,  1323,  1433,   159, 10114], dtype=int32)>

In [27]:
vocab_file

'/space/engineering/pretrained_models/bert/multi_cased_L-12_H-768_A-12/vocab.txt'

In [24]:
sp3.id_to_string([0,1,2,3])

<tf.Tensor: shape=(4,), dtype=string, numpy=array([b'<pad>', b'<unk>', b'[CLS]', b'[SEP]'], dtype=object)>

In [28]:
sp3.tokenize_with_offsets(['talk to an agent','talk with a representative'])

(<tf.RaggedTensor [[930, 20, 40, 2267], [930, 29, 21, 2652]]>,
 <tf.RaggedTensor [[0, 4, 7, 10], [0, 4, 9, 11]]>,
 <tf.RaggedTensor [[4, 7, 10, 16], [4, 9, 11, 26]]>)

In [29]:
sp3.tokenize(['talk to an agent','talk with a representative'])

<tf.RaggedTensor [[930, 20, 40, 2267], [930, 29, 21, 2652]]>

In [51]:
encoded_sentences = sp3.tokenize(['dummy test sentence','[CLS] test sentence try'])

In [53]:
encoded_sentences

<tf.RaggedTensor [[26660, 1289, 5123], [636, 1, 500, 1289, 5123, 1131]]>

In [52]:
encoded_sentences.to_tensor()

<tf.Tensor: shape=(2, 6), dtype=int32, numpy=
array([[26660,  1289,  5123,     0,     0,     0],
       [  636,     1,   500,  1289,  5123,  1131]], dtype=int32)>

In [ ]:
tf.cast()

In [34]:
get_encoded_text(tf.constant(['talk to an agent','talk to a representative']),128)

NameError: in user code:

    <ipython-input-31-e0231a979fdb>:37 None  *
        encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),input_text_batch,dtype=(tf.int32,tf.int32,tf.int32))
    <ipython-input-31-e0231a979fdb>:17 process_invidual_line_encoding  *
        concat = tf.concat([[self.CLS_ID],values_trimmed,[self.SEP_ID]],axis=0,name='concat_out')

    NameError: name 'self' is not defined


In [43]:
max_seq_length = 10
sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,max_seq_length=tf.constant(max_seq_length))
sentence_piece_layer.build(1)
sentence_piece_layer.call(tf.constant(['talk to an agent','talk to a representative']))

(<tf.Tensor: shape=(2, 10), dtype=int32, numpy=
 array([[   2,  930,   20,   40, 2267,    3,    0,    0,    0,    0],
        [   2,  930,   20,   21, 2652,    3,    0,    0,    0,    0]],
       dtype=int32)>,
 <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>,
 <tf.Tensor: shape=(2, 10), dtype=int32, numpy=
 array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], dtype=int32)>)

In [13]:

class SentencepieceTokenization(tf.keras.layers.Layer):
    def __init__(self,model_path,max_seq_length):
        super(SentencepieceTokenization, self).__init__(trainable=False,dtype=tf.int64)
        self.model_path = model_path
        # TODO keep a tf.constant else tracing issues 
        self.max_seq_length = max_seq_length
        
    def build(self,input_shape):
        self.model_proto = tf.io.gfile.GFile(self.model_path, 'rb').read()
        self.sp_tokenizer =  tftext.SentencepieceTokenizer(model=self.model_proto)
        ## the output from id_to_string can be used to validate CLS_ID, SEP_ID
        self.sp_tokenizer.id_to_string([0,1,2,3])
        ## pad_id is usually 0 
#         [self.CLS_ID,self.SEP_ID,self.PAD_ID]  = self.sp_tokenizer.tokenize(['[CLS]','[SEP]','<pad>'],model_file=self.model_path)
        [self.PAD_ID, self.CLS_ID, self.SEP_ID] = [0, 2, 3]
        self.built=True
        
# #     @tf.function    
#     def call(self, input_text):
#         ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
#         encoded_text = self.sp_tokenizer.tokenize(input_text).to_tensor()
#         return encoded_text
    
    
    def call(self, input_text):
        ##tensorflow sentence piece works while exporting to graph while, tf_text sentencepiece doesn't
        encoded_text = self.get_encoded_text(input_text,max_sequence_length=self.max_seq_length)
        return encoded_text

#     @tf.function
    def get_encoded_text(self,input_text_batch,max_sequence_length):
        def process_invidual_line_encoding(x):
#             tf sentencepiece requires a list as input, while the individual value that we get here
#             is a single sentence, so adding one more dimension (i.e adding batch dimension = 1)
            list_x = tf.expand_dims(x,axis=0)
            #sp_encoded = tfs.encode(list_x,model_proto=self.model_proto)
            sp_encoded = self.sp_tokenizer.tokenize(list_x)
    #         removing the batch dim with size=1 (which we added in the previous step)
            #values = tf.squeeze(sp_encoded.values,name='squeezed_values')
            values = tf.squeeze(sp_encoded.to_tensor(),name='squeezed_values')
            # casting down to int32 from int64 as subsequent operations required int32
            sequence_length = tf.squeeze(tf.cast(sp_encoded.row_lengths(),tf.int32))
    #         trimming to max_length-2 (-2 to incorporate [CLS], [SEP])
            trimmed_max_length = max_sequence_length-2
            values_trimmed = tf.cond(tf.greater(sequence_length,trimmed_max_length), 
                                    lambda : tf.slice(values,begin=[0],size=[trimmed_max_length],name='trimmed_out'),lambda : values)
            concat = tf.concat([[self.CLS_ID],values_trimmed,[self.SEP_ID]],axis=0,name='concat_out')
            # tf.shape gives dynamic shape &
            # tenorflow_variable.shape gives static shape with dynamic entries = None
            actual_token_length = tf.shape(concat)[-1]
            # need not prepend anything so 0 for 1st entry in padding, 
            # & next value for padding is how many dimensions required at the end of tensor
            padded = tf.pad(concat,paddings=[[0,max_sequence_length-actual_token_length]],name='input_ids')
    #         segment_ids = tf.zeros(shape=tf.shape(padded),dtype=tf.int32)
    #         or
            # This will create zeros with similar shape as padded tensor
            segment_ids = tf.zeros_like(padded,dtype=tf.int32,name='segment_ids')
            
            # This tensor is zero initialized & will update 0th index till actual_token_length with 1
            input_mask = tf.scatter_nd(indices=tf.expand_dims(tf.range(0,actual_token_length),axis=1),
                                       updates=tf.ones(shape=[actual_token_length],dtype=tf.int32),
                                       shape=[max_sequence_length],name='input_mask')
            return (padded,segment_ids,input_mask)
    #     Issue running map_fn on gpu https://github.com/tensorflow/tensorflow/issues/28007 
    #     https://www.tensorflow.org/api_docs/python/tf/device
#         with tf.device('/device:CPU:0'):
        encoded = tf.map_fn(lambda x: process_invidual_line_encoding(x),input_text_batch,dtype=(tf.int32,tf.int32,tf.int32))
        return encoded
    
    
    def get_config(self):
        config = super(SentencepieceTokenization, self).get_config()
        config.update({'model_path': self.model_path})
        return config

In [16]:
sentencepiece_path

b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'

In [18]:
print(f'sentencepiece_path {sentencepiece_path}')
sentencepiece_path = sentencepiece_path.decode('utf-8')
print(f"decoded sentencepiece_path {sentencepiece_path}")

sentencepiece_path b'/space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model'
decoded sentencepiece_path /space/engineering/tfhub_modules/d0ceaf43f67b8744561ebeeaea4c7c188a6e6f78/assets/30k-clean.model


In [19]:
max_seq_length = 128
input_text = tf.keras.Input(shape=(),dtype=tf.string,name='input_text')
sentence_piece_layer = SentencepieceTokenization(model_path=sentencepiece_path,max_seq_length=tf.constant(max_seq_length))
encoded_text = sentence_piece_layer(input_text)
model  = tf.keras.Model(inputs={'input_text':input_text},
                         #outputs={'encoded_text':encoded_text}
                        outputs=encoded_text
                       )

In [20]:
final_model_path_tf23='/space/engineering/tf_serve/models/tryout_jithin_sentencepiece_tf23'
model.save(os.path.join(final_model_path_tf23,'2'))

INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_sentencepiece_tf23/2/assets


INFO:tensorflow:Assets written to: /space/engineering/tf_serve/models/tryout_jithin_sentencepiece_tf23/2/assets
